In [0]:
# Google Hash
# Team The Hoddle Warriors
# Feb 20 2020
# Updated Feb 24 2020 - 1

import itertools
import time


##############
# Functions  #
##############


def total_books_given_library_id_and_days(id, days):
    books = libraries[id][0]
    first_shipping_day = libraries[id][1]
    daily_limit = libraries[id][2]
    remaining_days = days - first_shipping_day
    days_to_completion = books//daily_limit

    if remaining_days < 1:
        return 0

    if books % daily_limit != 0: # add one for ...
        days_to_completion += 1

    if days_to_completion <= remaining_days:
        return books
    else:
        return remaining_days * daily_limit

def all_pos_books_shipped_given_library_id_and_days(id, days):
    books_to_send = total_books_given_library_id_and_days(id, days)
    # will need to create dictionary with library id and days
    total_available_books = libraries[id][0]
    available_books = libraries[id][3:]


    print(f'books to send: {books_to_send}, total_available_books: {total_available_books}')
    # print(f'books to send: {books_to_send}')

    if books_to_send == 0:
        return []
    elif books_to_send == total_available_books:
        return available_books
    else:
        # return list of combinations
        # WARNING -- this method is going to make this impossible. Need a way to narrow down choices.
        # or rethink entire approach. For example, start with the highest producing library, then choose the
        # next one based on the remaining books.
        # would need a function that yields best_set_of_books given days and used_books (looping over all remaining libraries)
        temp = list(itertools.combinations(available_books, books_to_send))
        return temp

def get_score_given_books(books):
    total = 0
    for book in books:
        total += book_scores[book]
    return total

def highest_yielding_library_given_days_and_existing_books(book_set, days, set_of_library_ids):
    best_score = 0
    best_set = []
    best_id = -1
    new_best_flag = 0
    for id in set_of_library_ids:
        print(f' checking library id: {id} of {len(libraries)}')
        list_of_lists = all_pos_books_shipped_given_library_id_and_days(id, days)
        print(f'lists have been generated:')
        if not list_of_lists:
            continue
        if type(list_of_lists[0]) == int:
            list_of_lists = [list_of_lists]
        best_list = []
        for list_of_books in list_of_lists:
            pos = list(book_set | set(list_of_books))
            score = get_score_given_books(list_of_books)
            if score > best_score:
                best_id = id
                best_set = list_of_books
                best_score = score
    return best_id, best_set


# ABANDONED! This was my first attempt. I realized it was going to be impossible to make it small/work given my knowledge
def recursive_library_choice_get_max_ouput(days, set_of_library_ids):
    if days < 0:
        return 0, [], {}
    max_books = 0
    best_book_combination = []

    for id in set_of_library_ids:
        remaining_libraries = set_of_library_ids - {id}
        signup_time = library[id][1]
        new_possibilities = all_pos_books_shipped_given_library_id_and_days(id, days)
        dic_of_possibilities = recursive_library_choice_get_max_ouput(days - signup_time, remaining_libraries)
        dic_of_possibilities
        # existing_possibilities, new_possibilities
    return max_books


def simple_search():
    print(f'about to simple search...')
    full_library_set = set(range(0, LIBRARIES))
    remaining_libraries = full_library_set
    days_remaining = DAYS
    all_books_already_sent = set()
    while remaining_libraries and days_remaining > 0:
        print(f'days remaining: {days_remaining}')
        id, list_of_books = highest_yielding_library_given_days_and_existing_books(all_books_already_sent, days_remaining, remaining_libraries)

        if id == -1:
            # all libraries have been exhausted. nothing yields > 0
            return 0

        remaining_libraries = remaining_libraries - {id}
        all_books_already_sent = all_books_already_sent | set(list_of_books)

        days_remaining -= libraries[id][1]
        to_save_library_list.append(id)
        to_save_books.append(list(list_of_books))

    print(f' simple search concluded.')

def order_books_by_value(books):
    return 0

def num_books_shipped_given(id, books, days):
    num_books = len(books)
    first_shipping_day = libraries[id][1]
    daily_limit = libraries[id][2]
    remaining_days = days - first_shipping_day
    days_to_completion = num_books//daily_limit

    if remaining_days < 1:
        return 0

    if num_books % daily_limit != 0: # add one for ...
        days_to_completion += 1

    if days_to_completion <= remaining_days:
        return num_books
    else:
        return remaining_days * daily_limit



def get_best_books_given_used_books(id, viable_books, days_remaining):
    if id in dic_library_to_books_ordered_by_value:
        besties, score = dic_library_to_books_ordered_by_value[id]
        besties = [i for i in besties if i in viable_books]
    else:
        viable_books = viable_books & set(libraries[id][3:])
        besties = [i for i in books_by_value if i in viable_books]

    days_to_start = libraries[id][1] # ignore daily rate
    days_to_complete = len(besties)/libraries[id][2] + libraries[id][1]
    books_shipped = num_books_shipped_given(id, besties, days_remaining)

    besties = besties[:books_shipped]
    score = sum([dic_book_id_to_value[i] for i in besties]) / days_to_start
    dic_library_to_books_ordered_by_value[id] = (besties, score)

    # Consider the median delay time ?
    return set(besties), score


# imperfect, BOTTOM UP approach to creating best library
def initialize_library_set():
    viable_books = set(range(BOOKS))
    lib_stack = []
    list_of_book_lists = []
    rem_libraries = set(range(LIBRARIES))
    days_remaining = DAYS
    progress_pointer = 0

    while days_remaining > 0:
        # initialize
        best_score = 0
        best_id = -1
        best_books = []

        if((1 - days_remaining/DAYS)*100 > progress_pointer):
            print(f'{progress_pointer}% complete. {days_remaining} days remain of {DAYS}.')
            progress_pointer += 10 # started at one before

        if not rem_libraries:
            break

        # ToDo -- double stack search, as opposed to single
        for id in rem_libraries:
            # check if possible to beat best score. Move on if impossible to beat.
            # this could be made more functional. For now, it is just used for score checking
            if id in dic_library_to_books_ordered_by_value:
                besties, score = dic_library_to_books_ordered_by_value[id]
                besties = [i for i in besties if i in viable_books]
                if score <= best_score:
                    continue

            new_books, new_score = get_best_books_given_used_books(id, viable_books, days_remaining)
            if new_score > best_score:
                best_score = new_score
                best_id = id
                best_books = new_books
        if best_score != 0:
            lib_stack.append(best_id)
            viable_books = viable_books - best_books
            days_remaining -= libraries[best_id][1]
            rem_libraries = rem_libraries - {best_id}
            list_of_book_lists.append(best_books)
        else:
            days_remaining = -1

    print(f'100% complete. 0 days remain of {DAYS}.')

    return lib_stack, set(range(BOOKS)) - viable_books, list_of_book_lists


#########
# NOTES #
#########

# ToDo speed up initalize_library_set
# ToDo --Binary Union Exclusion
''' Split the libraries in half. Take the union for the books of the right.
Get the score of the books on the left based on the exclusion from the union.
Repeat for the right side, cutting it in half... There are a few options.
Start gain, but flip around and do the same?  Still brainstorming.'''


#########
# MAIN #
#########

# load filenames
FILENAMES = []
OUTFILENAMES = []
for c in "abcdef":
    FILENAMES.append(c + ".txt")
    OUTFILENAMES.append(c +".out")


total_score = 0
# Start main loop
for in_filename, out_filename in zip(FILENAMES, OUTFILENAMES):
    # load data
    initial_vals_list = []
    print(f'in_filename: {in_filename}')
    print(f'out_filename: {out_filename}')
        

    # Prep Data #

    # will use these to store data to print
    to_save_library_list = []
    to_save_books= []

    # load data
    with open(in_filename, 'r') as f:
        for line in f:
    #        for s in line.split(' '):
            initial_vals_list.append(line)

    BOOKS, LIBRARIES, DAYS = [int(c) for c in initial_vals_list[0].split()]
    initial_vals_list = initial_vals_list[1:]
    book_scores = [int(c) for c in initial_vals_list[0].split()]
    initial_vals_list = initial_vals_list[1:]
    libraries = []



    for i in range(0,LIBRARIES):
        # amount of books, signup_time, daily shipping limit
        # books

        temp_vals_0 = [int(c) for c in initial_vals_list[0].split()]
        temp_vals_1 = [int(c) for c in initial_vals_list[1].split()]
        libraries.append(temp_vals_0 + temp_vals_1)
        initial_vals_list = initial_vals_list[2:]

    print(f'\n----- Finished loading data -----\n')
    # print(libraries)


    # create a dictionary of book id to value
    dic_book_id_to_value = {i:score for i, score in enumerate(book_scores)}
    books_by_value = list(sorted(dic_book_id_to_value, key=dic_book_id_to_value.get, reverse=True))

    # {set of dic_id --> books}
    dic_of_libraries = {}
    # print(f' the best order for books is: {books_by_value}')

    dic_library_to_books_ordered_by_value = {}


    # Process Data #

    # create a dictionary of {library set --> book set, score
    lib_stack, used_books, list_of_book_lists = initialize_library_set()
    print(f'\n----- Finished initializing library set -----\n')

    # get score
    score = sum([book_scores[i] for i in used_books])
    total_score += score

    print(f'Results:')
    print(f'lib_stack: {lib_stack}, books amount: {len(used_books)}')
    print(f'Expected score: {score}')

    # Create Output
    to_save_library_list = lib_stack
    to_save_books = list_of_book_lists


    # OUTPUT  #

    # Create Output -- This could be removed if variables changed
    to_save_library_list = lib_stack
    to_save_books = list_of_book_lists

    with open(out_filename, 'w') as file:
        file.write(f'{len(to_save_library_list)}\n')
        for i in range(len(to_save_books)):
            id = to_save_library_list[i] #assume same length
            file.write(f'{id} {len(to_save_books[0])}\n')
            temp = [str(i) for i in to_save_books[0]]
            file.write(" ".join(temp) + "\n")
            to_save_books = to_save_books[1:]

print(f'\n----- Finished Everything -----\n')
print(f'Expected Total Score: {total_score}')


in_filename: a.txt
out_filename: a.out

----- Finished loading data -----

0% complete. 5 days remain of 7.
10% complete. 2 days remain of 7.
100% complete. 0 days remain of 7.

----- Finished initializing library set -----

Results:
lib_stack: [0, 1], books amount: 6
Expected score: 21
in_filename: b.txt
out_filename: b.out

----- Finished loading data -----

0% complete. 999 days remain of 1000.
10% complete. 898 days remain of 1000.
20% complete. 793 days remain of 1000.
30% complete. 695 days remain of 1000.
40% complete. 589 days remain of 1000.
50% complete. 499 days remain of 1000.
60% complete. 391 days remain of 1000.
70% complete. 292 days remain of 1000.
80% complete. 187 days remain of 1000.
90% complete. 97 days remain of 1000.
100% complete. 0 days remain of 1000.

----- Finished initializing library set -----

Results:
lib_stack: [38, 92, 27, 29, 48, 73, 77, 11, 34, 66, 88, 21, 32, 65, 75, 82, 18, 37, 57, 76, 2, 9, 22, 62, 14, 16, 31, 89, 13, 42, 46, 52, 53, 67, 69, 79, 